# Code Interpreter Agent

### Connect to the Foundry

In [8]:
#use foundrywus
from util import get_project_client
project_client = get_project_client()

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageRole,
    RunStepToolCallDetails,
    BrowserAutomationTool,
    RunStepBrowserAutomationToolCall,
)
from azure.identity import DefaultAzureCredential

connection_id = os.environ["AZURE_PLAYWRIGHT_CONNECTION_NAME"]

# Initialize Browser Automation tool and add the connection id
browser_automation = BrowserAutomationTool(connection_id=connection_id)

with project_client:

    agents_client = project_client.agents

    # Create a new Agent that has the Browser Automation tool attached.
    # Note: To add Browser Automation tool to an existing Agent with an `agent_id`, do the following:
    # agent = agents_client.update_agent(agent_id, tools=browser_automation.definitions)
    agent = agents_client.create_agent(
        model=os.environ["AZURE_DEPLOYMENT_NAME"],
        name="my-agent",
        instructions="""
            You are an Agent helping with browser automation tasks. 
            You can answer questions, provide information, and assist with various tasks 
            related to web browsing using the Browser Automation tool available to you.
            """,
        tools=browser_automation.definitions,
    )


    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content="""
            Your goal is to help me find a few laptops that best suite my work. I am a researcher and often train ML models. I am also in long meetings regularly.
            Go to Amazon.com
            At the top of the page, you will find a search bar.
            Run a few keyword searches to surface the best laptops for me.
            For each laptop you pick, get the brand, specs, price and  star rating.
            Compile a report with your recommendations.
            """,
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    print(f"Waiting for Agent run to complete. Please wait...")
    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch run steps to get the details of the agent run
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Step {step.id} status: {step.status}")

        if isinstance(step.step_details, RunStepToolCallDetails):
            print("  Tool calls:")
            tool_calls = step.step_details.tool_calls

            for call in tool_calls:
                print(f"    Tool call ID: {call.id}")
                print(f"    Tool call type: {call.type}")

                if isinstance(call, RunStepBrowserAutomationToolCall):
                    print(f"    Browser automation input: {call.browser_automation.input}")
                    print(f"    Browser automation output: {call.browser_automation.output}")

                    print("    Steps:")
                    for tool_step in call.browser_automation.steps:
                        print(f"      Last step result: {tool_step.last_step_result}")
                        print(f"      Current state: {tool_step.current_state}")
                        print(f"      Next step: {tool_step.next_step}")
                        print()  # add an extra newline between tool steps

                print()  # add an extra newline between tool calls

        print()  # add an extra newline between run steps

    # Optional: Delete the agent once the run is finished.
    # Comment out this line if you plan to reuse the agent later.
    agents_client.delete_agent(agent.id)
    print("Deleted agent")

    # Print the Agent's response message with optional citation
    response_message = agents_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
    if response_message:
        for text_message in response_message.text_messages:
            print(f"Agent response: {text_message.text.value}")
        for annotation in response_message.url_citation_annotations:
            print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Created agent, ID: asst_te9zs6K5Kz3QnSJzKyGff5TQ
Created thread, ID: thread_7kTbrt8hUgibamTTBhn3rMnM
Created message, ID: msg_dHnhF4kXFn338KkaPa2b494d
Waiting for Agent run to complete. Please wait...
Run finished with status: RunStatus.COMPLETED
Step step_Dl1TLp2twkEg4llqADo9oW6j status: RunStepStatus.COMPLETED

Step step_VraXGVo0gkGcj7agflKtx8tx status: RunStepStatus.COMPLETED
  Tool calls:
    Tool call ID: call_i0s5YnGLpQRQzBtFmFqTVEwt
    Tool call type: browser_automation
    Browser automation input: Go to https://amazon.com and perform multiple keyword searches in the search bar for laptops suited for machine learning researchers and heavy meeting use (e.g., "laptop for machine learning", "laptop for data science", "laptop with powerful GPU", "business laptop for research"). Identify top laptops and capture each one's brand, key specs, and star rating.
    Browser automation output: Here are the top laptops identified from Amazon for each of your requested keyword searches, inc

 Your goal is to get me the top 5 latest 8-k SEC fillings for Amazon. 
    Steps:
    1. Go to Amazon.com
    2. Find the About or Investor Relations link at the bottom of the page and click it.
    3. Select SEC Filings 
    4. In the Group dropdown select "Current"
    5. Collect the latest 5 filings, including the filing date and a link to the PDF.